In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, metrics

In [2]:
from scipy.ndimage import convolve
from scipy.signal import convolve2d

class Connect4Game(object):
  def __init__(self):
    self.board = np.zeros((6, 7))
    self.tops = np.zeros(7)
    self.moves_played = 0
    self.pieces = {1: "|0|",
                   0: "|·|",
                   -1:"|O|"}
  def render(self):
    for row in reversed(range(6)):
      print(''.join([self.pieces[i] for i in self.board[row].astype(np.int)]))
  def reset(self):
    self.board *= 0
    self.tops *= 0
    self.moves_played = 0
  def play(self, col, color):
    self.board[int(self.tops[col])][col] = color
    self.tops[col] += 1
    self.moves_played += 1
  def result(self):
    """ Returns 0 if it's a tie; returns None if the game is not over; returns -1 if player -1 wins, and 1 if player 1 wins."""
    if self.moves_played == 42: return 0
    kernel_row = [[1, 1, 1, 1]]
    kernel_col = [[1],
                  [1],
                  [1],
                  [1]]
    kernel_dia = [[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]]
    kernel_dia2 = [[0, 0, 0, 1],
                  [0, 0, 1, 0],
                  [0, 1, 0, 0],
                  [1, 0, 0, 0]]
    for kernel in [np.array(kernel_row), np.array(kernel_col), np.array(kernel_dia), np.array(kernel_dia2)]:
      check = convolve2d(self.board, kernel, mode='same', boundary='fill', fillvalue=0)
      #print(check)
      if (check == 4).any(): return 1
      check2 = convolve2d(self.board, -kernel, mode='same', boundary='fill', fillvalue=0)
      if (check2 == 4).any(): return -1
    return None 


In [3]:
def human_play():
  game = Connect4Game()
  turn = 1
  while game.result() == None:
    game.render()
    game.play(int(input()), turn); turn *= -1
  print("Result: ", game.result())

In [7]:
import tqdm
from tensorflow.keras.utils import plot_model
import tensorflow.keras.backend as kb

def create_P_network(NUM_CONV_LAYERS = 3, describe = True):
    in_ = layers.Input((6, 7))
    X = layers.Reshape((6, 7, 1))(in_)
    for i in range(NUM_CONV_LAYERS-1):
        X_skip = layers.Conv2D(32, (4,4), activation = 'relu', padding = "same")(X)
        X_skip = layers.BatchNormalization()(X_skip)
        X_skip = layers.Conv2D(32, (2,2), activation = 'relu', padding = "same")(X_skip)
        X_skip = layers.BatchNormalization()(X_skip)
        X = layers.BatchNormalization()(X)
        X = layers.Add()([X_skip, X])
    X = layers.Conv2D(32, (3,3), activation = 'relu')(X)
    X = layers.BatchNormalization()(X)
    #X = layers.GlobalAveragePooling2D()(X)
    X = layers.Flatten()(X)
    #X = layers.Dense(256, activation = 'relu')(X)
    X = layers.Dense(7, activation = 'softmax')(X)
    model = models.Model(in_, X)
    if describe: 
        model.summary()
    return model
            
# All a class needs to be a valid player is the method make_move(self, state, tops) and the attribute name.

class EvoPlayer(object):
    def __init__(self, model, name):
        self.model = model; self.name = name
        self.DETERMINISTIC = True
    def norm_probs(self, probs, tops):
        probs *= (tops != 6)
        probs *= (1 / np.sum(probs))
        return probs
    def make_move(self, state, tops): # Tops is the top of each column in game, used for not making invalid moves
        Q = self.model(state[np.newaxis,:,:]) + 0.0001
        probs = Q.numpy()[0]
        probs = self.norm_probs(probs, tops)
        if not self.DETERMINISTIC: action = np.random.choice(range(0, 7), p = probs)
        else: action = np.argmax(probs)
        return action
    
class RandomPlayer(object):
    def __init__(self, name = "Random"):
        self.name = name
    def norm_probs(self, probs, tops):
        probs *= (tops != 6)
        probs *= (1 / np.sum(probs))
        return probs
    def make_move(self, state, tops):
        probs = np.ones(7) / 7
        probs = self.norm_probs(probs, tops)
        return np.random.choice(range(0, 7), p = probs)

# Evaluate
import random

class Ladder(object):
    def __init__(self):
        self.game = Connect4Game()
    def play_match(self, player_1, player_neg1, render = False):
        turn = 1
        self.game.reset()
        while (self.game.result() == None):
            if render: self.game.render()

            state = self.game.board * turn
            if turn == 1:
                action = player_1.make_move(state, self.game.tops)
            if turn == -1:
                action = player_neg1.make_move(state, self.game.tops)
            self.game.play(action, turn)
            
            turn *= -1
            
        result = self.game.result()
        self.game.reset()
        return result
    
    def play_tournament(self, players, NUM_MATCHES = 1000, K = 64, onesd = 400):
        MMR = {}
        for player in players:
            MMR[player.name] = 1000
        for match in tqdm.trange(NUM_MATCHES):
            p1, p_neg1 = random.sample(players, 2)
            result = self.play_match(p1, p_neg1)
            
            SA = (result + 1) / 2 # A scores 1 point if win, 0 points if loss.
            SB = (-result + 1) / 2 # B scores 1 point if win, -1 points if loss.
            # Both score 0.5 points if tie.
            EA = 1 / (1 + 10**((MMR[p_neg1.name] - MMR[p1.name]) / onesd))
            EB = 1 / (1 + 10**((MMR[p1.name] - MMR[p_neg1.name]) / onesd))
            
            MMR[p1.name] = MMR[p1.name] + K * (SA - EA)
            MMR[p_neg1.name] = MMR[p_neg1.name] + K * (SB - EB)
            
        return MMR
    
class EvoLearner(object):
    def __init__(self):
        self.pop_size = 30
        self.mutation_rate = 0.1
        self.pop = [create_P_network(NUM_CONV_LAYERS = 2, describe = False) for i in range(self.pop_size)]
        self.ladder = Ladder()
        
    def mutate_pop(self, scores):
        sum_scores = sum([scores[i] for i in scores])
        new_weights = [np.zeros_like(weights) for weights in self.pop[0].get_weights()]
        
        for i in scores:
            for index, weights in enumerate(self.pop[i].get_weights()):
                new_weights[index] += (weights * scores[i] / sum_scores)
        
        for i in self.pop:
            ind_weights = []
            for weight in new_weights:
                ind_weights.append(weight + (np.random.normal(size = weight.shape) * self.mutation_rate * (1+weight)) )
            i.set_weights(ind_weights)
        
    def train(self, generations = 1000, initial_players = [], save_benchmark_every = 10, prop_NEGATED = 0, verbose = True):
        train_history = []
    
        players = initial_players
        players.extend([EvoPlayer(model, index) for index, model in enumerate(self.pop)])
        
        for i in tqdm.trange(generations):
            for player in players:
                if (type(player.name) == int): 
                    player.model = self.pop[player.name]
                    
            MMR = self.ladder.play_tournament(players, NUM_MATCHES = 500)
            results = {}
            for key in MMR:
                results[key] = 1000 + MMR[key] - MMR["Random"]
            if verbose: 
                print({k: v for k, v in sorted(results.items(), key=lambda item: item[1])})
                print("Mean score: ", sum([results[k] for k in results]) / len(results))
            train_history.append(results)
            
            invalid_idxs = [k for k in MMR if type(k) != int]
            for idx in invalid_idxs:
                MMR.pop(idx)
                
            min_score = min([MMR[k] for k in MMR])
            scores = {}
            for key in MMR:
                scores[key] = MMR[key] - min_score
            sum_scores = sum([scores[k] for k in scores])
            mean_score = sum_scores / len(scores)
            evo_scores = {}
            for key in scores:
                evo_scores[key] = scores[key] - mean_score * prop_NEGATED
                
            max_score = max([MMR[k] for k in MMR])
            for k in MMR:
                if MMR[k] == max_score: max_ind = k
                    
            if i % save_benchmark_every == 0:
                players.append(EvoPlayer(self.pop[max_ind], "Best of generation " + str(i)))
            self.mutate_pop(evo_scores)
        
        return train_history

In [8]:
population = EvoLearner()

In [9]:
initial_players = [RandomPlayer()]

hist = population.train(generations = 100, initial_players = initial_players)

 25%|████████████████████▌                                                             | 125/500 [00:10<00:33, 11.20it/s]


 51%|██████████████████████████████████████████▏                                       | 257/500 [00:20<00:21, 11.44it/s]


 72%|███████████████████████████████████████████████████████████                       | 360/500 [00:32<00:12, 11.66it/s]


 96%|███████████████████████████████████████████████████████████████████████████████   | 482/500 [00:42<00:01, 10.72it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 500/500 [00:44<00:00, 11.20it/s]


...

 87%|███████████████████████████████████████████████████████████████████████▏          | 434/500 [00:30<00:05, 11.48it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 500/500 [00:34<00:00, 14.41it/s]


{3: 775.2709877574051, 8: 836.8523143822446, 18: 875.298044200582, 12: 919.4850320277584, 'Best of generation 0': 942.7166291349746, 26: 954.6219682592069, 2: 978.5852376692854, 21: 981.9110762872589, 24: 997.4682089798226, 'Random': 1000.0000000000001, 'Best of generation 20': 1026.7933620911085, 4: 1036.1536163867659, 25: 1037.4127035626452, 0: 1048.421473676026, 22: 1048.661586573519, 16: 1088.708008719519, 1: 1093.1329414072588, 9: 1106.017545323757, 17: 1122.2036002000918, 23: 1133.512938703448, 15: 1141.4065063699059, 5: 1144.6964227132794, 10: 1147.8073105881094, 13: 1152.779009767491, 7: 1156.4153753355145, 14: 1157.2881457960689, 29: 1160.454816104902, 11: 1175.2431836055516, 27: 1180.9836124345707, 'Best of generation 30': 1190.631377884852, 'Best of generation 10': 1196.3452321796099, 28: 1226.605039525511, 19: 1247.085843448614, 20: 1320.500129723458, 6: 1360.119365813382}
Mean score:  1084.6168184752423


 36%|█████████████████████████████▉                                                     | 36/100 [20:54<37:09, 34.84s/it]


IndexError: index 6 is out of bounds for axis 0 with size 6

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
data = {'x':[],
       'y':[],
       'p':[]}
for j in range(25,100,25):
    data['y'].extend([np.percentile([i[k] for k in i], j) for i in hist])
    data['p'].extend([j] * len(hist))
    data['x'].extend(list(range(len(hist))))
data = pd.DataFrame(data)
sns.lineplot(data = data, x = "x", y = "y", hue = "p")
plt.plot(list(range(len(hist))), [1000] * len(hist), label="Random Agent")
plt.legend()
plt.show()

NameError: name 'hist' is not defined